### Importamos librerias

In [1]:
import pandas as pd
import spacy
import re
import emoji
from sklearn.preprocessing import LabelEncoder
nlp = spacy.load('es_core_news_sm')

/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Cargamos los datos en bruto

In [2]:
politicES_train = pd.read_csv('./Corpus/pratice_data/politicES_phase_2_train_public.csv')
print(politicES_train.shape)

(180000, 6)


In [3]:
politicES_train.head(3)

,label,gender,profession,ideology_binary,ideology_multiclass,tweet
0,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,¡Feliz 28 de febrero a todas las andaluzas y a...
1,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,"Feliz año nuevo, feliz esperanza 💕. Querido 20..."
2,0008c4fab9e97623a60380ee9c88cb20,female,politician,left,left,🇩🇪🇪🇸 ¡Un placer encontrarme con mi homólogo al...


### Reformateamos los datos

#### Normalizamos el texto

In [10]:
# Función que normaliza el texto, según una lista de etiquetas dada. 

def apply_regex(text:str):
    regexUser = re.compile("@+\w{1,15}") 
    regexHashtag = re.compile("#+\w+") 
    regexHour = re.compile("\d{1,2}[h]|\d{1,2}[:]\d{1,2}") 
    regexDate = re.compile("\[\d{1,2}/\d{1,2}") 
    regexNumber = re.compile("\d+")
    regexBrackets = re.compile('\[\w+\]') 
    regexAnd = re.compile('[yo]')
    regexList = [regexUser,regexHashtag,regexHour,regexDate,regexNumber,regexBrackets,regexAnd]

    valid = True
    for regex in regexList:
        if re.match(regex, text) or emoji.emoji_count(text) >= 1:
             valid = False
             break
    return valid # Devuelve True si la cadena es valida


def normalize(text:str): 
    regex_text = ""
    normalized_text = []
    # Usando Regex
    for token in text.split(' '):
        if apply_regex(token):
            regex_text += token + " "
    # Usando Spacy
    tags =  ('PUNCT', 'ADP', 'CONJ', 'SCONJ', 'PROP', 'DET')
    doc = nlp(regex_text)
    for token in doc:
            # Se verifica que el token no sea alguna etiqueta dada. 
                if token.tag_ not in tags: 
                    normalized_text.append(token.lemma_)
    # Se regresa el texto normalizado
    _text = ""
    for word in normalized_text:
        _text += word + " "
    _text = re.sub(' +', ' ', _text)
    return _text.strip().replace('\'','').replace('\"','').lower()

# ADP : adposition = preposicion
# CONJ : coordinating conjunction = conjuncion coordinante
# SCONJ :  subordinating conjunction = conjuncion subordinante
# PROP : pronoun = pronombre
# DET : determiner = articulo

normaliced_data = pd.DataFrame()

tweets_normaliced = []
for text in politicES_train['tweet']:
    tweets_normaliced.append(normalize(text))

normaliced_data['tweets'] = tweets_normaliced

#### Aplicando LabelEncoding para 'Genero', 'Profesion', 'Ideologia binaria', 'Ideologia Multiclase'

In [11]:
le = LabelEncoder()

normaliced_data['genero'] = le.fit_transform(politicES_train['gender'])
normaliced_data['profesion'] = le.fit_transform(politicES_train['profession'])
normaliced_data['ideologia_binaria'] = le.fit_transform(politicES_train['ideology_binary'])
normaliced_data['ideologia_multiclase'] = le.fit_transform(politicES_train['ideology_multiclass'])

normaliced_data.head(10)

,tweets,genero,profesion,ideologia_binaria,ideologia_multiclase
0,feliz febrero andaluza andaluz andalucía donde...,0,2,0,0
1,feliz año nuevo feliz esperanzo querido allá ir,0,2,0,0
2,placer encontrar yo homólogo alemán compartir ...,0,2,0,0
3,conflicto ucrania haber suponer dramático para...,0,2,0,0
4,academia llingua asturiana realizar trabajo gr...,0,2,0,0
5,cumplir ley derecho laboral no ser ser,0,2,0,0
6,placer haber poder conocer comunidad español u...,0,2,0,0
7,contratación indefinido comenzar ser norma red...,0,2,0,0
8,amazon no tener sindicato ser verdadero notici...,0,2,0,0
9,hoy detallar rueda prensa aprobación primero p...,0,2,0,0


In [12]:
print(normaliced_data.shape)
normaliced_data.to_csv('./Corpus/normaliced_data/politicES.csv')

(180000, 5)


In [6]:
print(politicES_train['gender'].unique())
print(politicES_train['profession'].unique())
print(politicES_train['ideology_binary'].unique())
print(politicES_train['ideology_multiclass'].unique())

['female' 'male']
['politician' 'journalist' 'celebrity']
['left' 'right']
['left' 'right' 'moderate_left' 'moderate_right']
